In [1]:
import sys, os
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms

In [2]:
sys.path.append("../")

In [3]:
from Transformers import UNetDataset, ChannelsFirst, ToTensor, Rescale

In [4]:
DATA_DIR_DEEPTHOUGHT="/storage/yw18581/data"
data_dir = DATA_DIR_DEEPTHOUGHT
train_test = os.path.join(data_dir, "train_validation_test")

In [5]:
data = np.load("/storage/yw18581/data/train_validation_test/Xy_train+val_clean_300_24_10_25.npz")
x = data["x"]
y = data['y']
dist = data['dist']

In [6]:
composed = transforms.Compose([Rescale(.25), ChannelsFirst(), ToTensor()])

In [7]:
train_dataset = UNetDataset(x, y, transform=composed, dist = dist)

In [8]:
data_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

In [9]:
torch.cuda.empty_cache() 

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
from cUNet_pytorch_pooling import cUNet, dice_loss

In [12]:
model = cUNet()

In [13]:
criterion_mask = dice_loss

In [ ]:
criterion_class = cross_entropy

In [15]:
model.to(device)

cUNet(
  (conv_block_down1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (downsampling_conv_1): Sequential(
    (0): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU(inplace=True)
  )
  (conv_block_down2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (downsampling_conv_2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU(inplace=True)
  )
  (conv_block_down3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (downsampling_conv_3): Sequential(
    (0): Co

In [14]:
import torch.optim as optim


optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [17]:
for epoch in range(10):
    print("Epoch {}\n".format(epoch+1))
    running_loss = 0.0
    for i, batch in enumerate(data_loader):
        inputs = batch['image'].float().to(device)
        labels_mask = batch['mask'].float().to(device)
        labels_dist = batch['dist'].float().to(device)
        
        optimizer.zero_grad()
        out_mask, out_class  = model(inputs)
        loss_mask = criterion_mask(out_mask, labels_mask)
        loss_class = criterion_class(out_class, labels_dist)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 2000 mini_batches
            print('[%d, %5d] loss: %.3f' % 
                 (epoch + 1, i + 1, running_loss / 30))
print('Finished Training')

Epoch 1

convb1: torch.Size([16, 16, 350, 350])
downsampling1: torch.Size([16, 16, 175, 175])
convb2: torch.Size([16, 32, 175, 175])
downsampling2: torch.Size([16, 32, 87, 87])
convb3: torch.Size([16, 64, 87, 87])
downsampling3: torch.Size([16, 64, 43, 43])
convb4: torch.Size([16, 128, 43, 43])
downsampling4: torch.Size([16, 128, 21, 21])
convb5: torch.Size([16, 256, 21, 21])
flatten: torch.Size([16, 112896])
fc : torch.Size([16, 4])
up6: torch.Size([16, 128, 21, 21])


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 43 and 21 in dimension 2 at /opt/conda/conda-bld/pytorch_1565287148058/work/aten/src/THC/generic/THCTensorMath.cu:71